# Sparse Neural Search

결과적으로 한글 지원 안됨!!!


## 1. 필요한 패키지 설치


In [1]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3

In [59]:
# Read data/movies.csv and save it in pandas dataframe
import os
import pandas as pd

movies_raw_df = pd.read_csv("./data/movies.csv", low_memory=False)
movies_raw_df.head(20)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url,keywords,plot_preprocessed_kmr,plot_preprocessed_kkma
0,1917,드라마|전쟁,2020,2.19,8.88,5017,제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에...,조지 맥케이|딘-찰스 채프먼,콜린 퍼스|베네딕트 컴버배치|마크 스트롱|앤드류 스캇|리차드 매든|에드리언 스카보로...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200212_161/1...,"['영국군', '중령', '대전', '한창', '공격', '병사', '부대', '수...",세계 대전 한창 독일군 통신망 파괴 상황 속 영국군 병사 딘 찰스 미션 함정 영국군...,세계 대전 한창 독일군 모든 통신망 파괴 상황 속 영국군 병사 채프먼 하나 미션 함...
1,21 브릿지: 테러 셧다운,범죄|드라마|스릴러,2020,1.01,7.84,579,"뉴욕 맨해튼 중심에서 벌어진 경찰 연쇄 살해 사건, 범인을 잡기 위해 베테랑 경찰 ...",채드윅 보스만|J.K. 시몬스|시에나 밀러|테일러 키취,스테판 제임스|빅토리아 카타게나|키스 데이빗|샤이나 라이언|알렉산더 시디그|모로코 ...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191227_205/1...,"['연쇄', '범인', '잡기', '베테랑', '사건', '수단', '방법', '극...",뉴욕 맨해튼 중심 경찰 연쇄 살해 사건 범인 잡기 베테랑 경찰 극단 조치 시간 수단...,뉴욕 맨해튼 중심 경찰 연쇄 살해 사건 범인 베테랑 경찰 극단 조치 시간 수단 방법 놈
2,n번째 이별중,코미디|멜로/로맨스,2020,4.01,6.48,177,여자친구 ‘데비’에게 대차게 차여 충격받은 물리학 천재 ‘스틸먼’. 사랑하는 그녀의...,에이사 버터필드|소피 터너|스카일러 거손도,오브리 레이놀즈|윌 펠츠|질리안 조이|케이든 J. 그레고리,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200409_56/15...,"['사랑', '플로', '완벽', '가동', '후회', '천재', '충격', '물리...",여자친구 충격 물리학 천재 사랑 마음 두뇌 풀 가동 후회 순간 타임머신 을 개발 안...,여자 친구 차여 충격 물리학 천재 사랑 마음 두뇌 가동 후회 순간 타임머신 개발 때...
3,가슴 큰 태희,멜로/로맨스,2020,2.27,8.67,3,곧 결혼을 앞둔 예비 신혼 부부 차욱과 민주. 차욱은 직장 후배인 용우네 부부에게 ...,NaN,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200211_88/15...,"['민주', '여행', '지연', '부부', '신혼', '제안', '사라', '예비...",결혼 예비 신혼 부부 차 민주 차 직장 후배 용 부부 커플 여행 제안 지연 불편 여...,결혼 예비 신혼 부부 차 과 민주 차 직장 후배 용 우 부부 커플 여행 제안 지연 ...
4,개인교습,멜로/로맨스,2020,3.02,10.00,1,부잣집 프랑스 아가씨 마고에게 첫눈에 반한 발리의 가난한 어부 에카는 그녀에게 피아...,하리 산티카|도르카스 카핀,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200226_62/15...,"['첫눈', '피아노', '부잣집', '프랑스', '아가씨', '마고', '서로',...",부잣집 프랑스 아가씨 마고 첫눈 발리 가난 어부 에카 피아노 개인 교습 부탁 건반 ...,부잣집 프랑스 아가씨 마 첫눈 발리 가난 어부 카 피아노 개인 교습 부탁 건반 사람...
5,갱,액션,2020,1.16,5.99,105,교도소를 개조해 만든 국내 최악의 고등학교 ‘대훈고’ 그리고 그 곳에 온 강제전학생...,차지혁|조선기|옥윤중|백재민|김대한|이정현,김라희|송은석|진성|이재한|맹재렬|이채성|최준민|시윤|지남혁|임정옥|권오윤|김동우|...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191115_141/1...,"['개조', '국내', '학생', '강제', '전학생', '목표', '배틀', '시...",교도소 개조 국내 최악 고등학교 곳 강제 전학생 최지훈 목표 현실 종소리 매점 대신...,교도소 개조 국내 최악 고등 학교 곳 강제 전 학생 지 훈 목표 단 이 현실 종 소...
6,건즈 아킴보,액션,2020,4.15,6.39,173,"낮엔 평범한 직장인, 밤엔 키보드 워리어 ‘마일즈’ 손가락 한번 잘못 놀렸다가 살인...",다니엘 래드클리프|사마라 위빙|네드 데네히|나타샤 류 보르디초,라이스 다비|재클린 리 거츠|콜린 모이,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200323_116/1...,"['게임', '시작', '구역', '플레이어', '설상가상', '관람', '익스트림...",낮 직장인 밤 키보드 워리어 손가락 살인 미션 레알 목숨 개념 게임 양손 권총 게임...,낮 직장인 밤 키보드 손가락 살인 미션 레 목숨 신 개념 게임 양손 권총 게임 강제...
7,걸즈 앤 판처 제 63회 전차도 전국 고교생 대회,애니메이션,2020,4.09,4.67,3,여성의 소양 중 하나로 손꼽히며 전통적인 무예의 교양 활동인 전차도로 유명한 니시즈...,후치가미 마이|카야노 아이|오자키 마미|이구치 유카,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200330_55/15...,"['전차', '참가', '전학', '도로', '교양', '활동', '폐교', '최종...",여성 소양 하나로 전통 무예 교양 활동 전차 도로 니 시즈 미 집안 전차 도가 아라...,여성 소양 전통적 무예 교양 활동 전차 도로 유명 시 미 집안 전차 라이 여 학원 ...
8,걸즈 앤 판처 최종장,애니메이션,2020,3.12,8.93,15,대학 선발팀과의 시합에서 우승하며 오아라이 여고를 폐교 위기에서 구해낸 미호와 전차...,후치가미 마이|카야노 아이|오자키 마미|이구치 유카|나카가미 이쿠미,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200303_117/1...,"['모모', '아라이', '여고', '우승', '경기', '전차', '자유', '고...",대학 선발팀 시합 우승 아라이 여고 폐교 위기 미 전차 팀원 선배 모모 유급 소식 ...,대학 선발팀 시합 우승 라이 여고 폐교 위기 미 호 전차 팀원 선배 모 모의 유급 ...
9,걸즈 앤 판처 최종장 제1화,애니메이션|드라마,2020,1.08,8.67,42,대학 선발팀과의 시합에서 우승하며 오아라이 여고를 폐교 위기에서 구해낸 미호와 전차...,후치가미 마이|카야노 아이|이구치 유카|오자키 마미|나카가미 이쿠미,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191231_269/1...,"['모모', '아라이', '여고', '우승', '경기', '전차', '자유', '고...",대학 선발팀 시합 우승 아라이 여고 폐교 위기 미 전차 팀원 선배 모모 유급 소식 ...,대학 선발팀 시합 우승 라이 여고 폐교 위기 미 호 전차 

In [ ]:
# Remove unnecessary columns
movies_columns_removed_df = movies_raw_df.loc[:, "title":"img_url"]
movies_columns_removed_df.head(5)

In [ ]:
# Nan 모두 제거
final_df = movies_columns_removed_df.fillna("정보없음")

In [5]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3
import json

In [6]:
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region="us-east-1")

aos_host = "search-bedrock-opensearch-6gczufsfv5pp76bvtsd62qkrxu.aos.us-east-1.on.aws"

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

### Pre-trained Model 등록(Register)

Sparse Nerual Search에 사용되는 모델들은 아래와 같은 DSL을 사용해 OpenSearch 안에 등록할 수 없다. 자세한 내용은 아래 OpenSearch Forun 링크를 참조하면 된다.

-   [OpenSearch Forum](https://forum.opensearch.org/t/how-to-register-sparse-encoding-model-in-aws-opensearch/17112)

Sparse Nerual Search 모델은 반드시 SageMaker에 Remote 형태로 배포해 OpenSearch에 Connector를 만들어 사용해야 하며, 이는 콘솔의 Integration을 사용하면 된다.

-   [Using AWS CloudFormation to set up remote inference for semantic search](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/cfn-template.html#cfn-template-sm)

Integration을 사용하면 CloudFormation Template을 통해 배포하게 되며, 완료되면 사용할 수 있는 Model ID를 Output에서 확인할 수 있다.


#### 사전 필수 작업

1. SageMaker Endpoint Connector를 등록할 수 있도록 url regex를 등록해준다.


In [ ]:
payload = {
    "persistent": {
        "plugins.ml_commons.trusted_connector_endpoints_regex": [
            "^https://runtime\\.sagemaker\\..*[a-z0-9-]\\.amazonaws\\.com/.*$",
            "^https://api\\.openai\\.com/.*$",
            "^https://api\\.cohere\\.ai/.*$",
            "^https://bedrock-runtime\\..*[a-z0-9-]\\.amazonaws\\.com/.*$",
        ]
    }
}
s = json.dumps(payload)
aos_client.cluster.put_settings(body=s)

2. `LambdaInvokeOpenSearchMLCommonsRole`의 Arn을 `ml_full_acess` Role의 Backend Role로 등록한다.

```sh
        arn:aws:iam::<account-id>:role/arn:aws:iam::account-id:role/role-name
```


3. Integration 메뉴에서 Neural Sparse Search를 위한 CloudFormation을 배포하고 필요한 정보를 가져온다.


In [49]:
model_id = "2PRegI4BnEOV7ezV88ah"

# nsr: Nerual Sparse Retrieval
nsr_pipeline = {
    "description": "An example neural search pipeline",
    "processors": [
        {
            "sparse_encoding": {
                "model_id": model_id,
                "field_map": {"title": "title_embedding", "plot": "plot_embedding"},
            }
        }
    ],
}
pipeline_id = "nsr_pipeline"
aos_client.ingest.put_pipeline(id=pipeline_id, body=nsr_pipeline)

{'acknowledged': True}

In [50]:
# Pipeline 생성 확인
aos_client.ingest.get_pipeline(id=pipeline_id)

{'nsr_pipeline': {'description': 'An example neural search pipeline',
  'processors': [{'sparse_encoding': {'model_id': '2PRegI4BnEOV7ezV88ah',
     'field_map': {'title': 'title_embedding', 'plot': 'plot_embedding'}}}]}}

In [51]:
movies_nsr_index = {
    "settings": {"default_pipeline": pipeline_id},
    "mappings": {
        "properties": {
            "date": {"type": "float"},
            "genre": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "img_url": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "main_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "page_url": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "plot": {"type": "text"},
            "rating": {"type": "float"},
            "supp_act": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "title": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "vote_count": {"type": "long"},
            "year": {"type": "long"},
            "title_embedding": {"type": "rank_features"},
            "plot_embedding": {"type": "rank_features"},
        }
    },
}

In [52]:
# aos_client.indices.delete(index="nsr_movies")
aos_client.indices.create(index="nsr_movies", body=movies_nsr_index)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'nsr_movies'}

### Ingest Data


In [53]:
payload = {
    "title": "파묘",
    "genre": "공포",
    "image_url": "http://pamyo.com",
    "page_url": "http://pamyo.com",
    "main_act": "최민식, 류해진",
    "plot": "아주 무서운 이야기",
    "rating": "10.0",
    "supp_act": "안기범, 권혁수",
    "vote_count": "100",
    "year": "2024",
}

aos_client.index(index="nsr_movies", body=payload, id=1)

{'_index': 'nsr_movies',
 '_id': '1',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 3, 'successful': 3, 'failed': 0},
 '_seq_no': 0,
 '_primary_term': 1}

In [69]:
query = {
    "size": 20,
    "query": {
        "neural_sparse": {
            "plot_embedding": {"query_text": "공포 영화", "model_id": "2PRegI4BnEOV7ezV88ah"}
        }
    },
}
res = aos_client.search(index="nsr_movies", body=query)
query_result = []
for hit in res["hits"]["hits"]:
    row = [
        hit["_id"],
        hit["_score"],
        hit["_source"]["title"],
        hit["_source"]["plot"],
        hit["_source"]["genre"],
        hit["_source"]["rating"],
        hit["_source"]["title_embedding"],
        hit["_source"]["plot_embedding"],
    ]
    query_result.append(row)

query_result_df = pd.DataFrame(
    data=query_result,
    columns=[
        "_id",
        "_score",
        "title",
        "plot",
        "genre",
        "rating",
        "title_embedding",
        "plot_embedding",
    ],
)
display(query_result_df)
display(res["hits"]["hits"][0]["_source"]["plot_embedding"])

,_id,_score,title,plot,genre,rating,title_embedding,plot_embedding
0,1,12.000269,파묘,아주 무서운 이야기,공포,10.0,"{'##ᅡ': 1.5833185, 'copyright': 0.06823575, '#...","{'country': 0.016660368, 'bus': 0.088327855, '..."


{'country': 0.016660368,
 'bus': 0.088327855,
 'symbol': 0.14296249,
 '##ᅡ': 1.3128623,
 'copyright': 0.38058123,
 '##ᅢ': 0.31632078,
 'japan': 0.08244367,
 '##ᅥ': 1.034421,
 'ᄆ': 0.4038362,
 '##ja': 0.8985563,
 'bae': 0.04136872,
 'political': 0.21451266,
 '##ᅩ': 0.25939262,
 'language': 0.079798535,
 'ᄋ': 1.2053391,
 'ᄌ': 0.3405793,
 'lee': 0.15044685,
 '##n': 0.049882386,
 '##ᅮ': 1.1500032,
 'jong': 0.12513442,
 'chinese': 0.49834573,
 '##ᅳ': 0.1587092,
 'ma': 0.0272439,
 'han': 0.42384976,
 'karen': 0.12534982,
 'sweden': 0.10698273,
 '##ᅵ': 0.36205596,
 'won': 0.42831406,
 'thailand': 0.004886115,
 'un': 0.2554292,
 '##eo': 0.22606896,
 'dance': 0.049708176,
 'tower': 0.084801495,
 'song': 0.63198835,
 '!': 0.5424492,
 'bible': 0.39696553,
 'mo': 0.0687487,
 'mp': 0.052289195,
 'in': 0.28248057,
 'king': 0.053591676,
 'festival': 0.38806194,
 '##ᄆ': 0.19481641,
 '(': 0.0058346908,
 'vietnam': 0.2734207,
 '##ᄉ': 0.7782851,
 'it': 0.044496875,
 'protest': 0.103246465,
 '##ᄋ': 0.7903

### References

-   https://qdrant.tech/articles/sparse-vectors/
